In [39]:
import requests
# uncomment this if you need to install the packages
# !{sys.executable} -m pip install pyarrow
import polars as pl

import time
email = str(open("info.txt", "r").read())
headers = {"User-Agent": email}

tickers_cik = requests.get("https://www.sec.gov/files/company_tickers.json", headers=headers)
time.sleep(.1)
df = pl.DataFrame(tickers_cik.json())

# ugly json flattening in regular python sorry :_( 
ciks, tiks, title = [], [], []
for i in df.iter_rows():
    for data in i:
        # fix ragged cik nums
        temp = str(data["cik_str"])
        temp = "0" * (10 - len(temp)) + temp
        ciks.append(temp)
        tiks.append(data["ticker"])
        title.append(data["title"])

# better dataframe
reformatted = pl.DataFrame( {
    "cik_str":ciks,
    "ticker":tiks,
    "name":title}
    )

In [40]:
# get all the historical data we want here!
# change this to whatever ticker you want to investigate
lookup = "MEDP"

lookup_cik, _, stock_name =reformatted.row(by_predicate=(pl.col("ticker")==lookup))

# edit the xBrl tags as needed
net_income_req = f"https://data.sec.gov/api/xbrl/companyconcept/CIK{lookup_cik}/us-gaap/NetIncomeLoss.json"
# NetCashProvidedByUsedInOperatingActivities
# CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents
cash_flow_req = f"https://data.sec.gov/api/xbrl/companyconcept/CIK{lookup_cik}/us-gaap/NetCashProvidedByUsedInOperatingActivities.json"
# RevenueFromContractWithCustomerExcludingAssessedTax
# RevenuesTotal
revenue_req = f"""https://data.sec.gov/api/xbrl/companyconcept/CIK{lookup_cik}/us-gaap/
                RevenueFromContractWithCustomerExcludingAssessedTax.json"""

net_inc = requests.get(net_income_req, headers=headers)
time.sleep(.1)
cash_flow = requests.get(cash_flow_req, headers=headers)
time.sleep(.1)
revenue = requests.get(revenue_req, headers=headers)
time.sleep(.1)

In [94]:
# uh stack overflow??? brain flattened 0.o
def flatten(kv, prefix=[]):
    for k, v in kv.items():
        if isinstance(v, dict):
            yield from flatten(v, prefix+[str(k)])
        else:
            if prefix:
                yield '_'.join(prefix+[str(k)]), v
            else:
                yield str(k), v

income_ts = net_inc.json()

income_pl = pl.DataFrame({k:v for k, v in flatten(kv)} for kv in income_ts["units"]["USD"])


income_pl = income_pl.with_columns(
    [pl.col("start").str.strptime(pl.Datetime, fmt="%Y-%m-%d").cast(pl.Datetime),
    pl.col("end").str.strptime(pl.Datetime, fmt="%Y-%m-%d").cast(pl.Datetime),
    pl.col("filed").str.strptime(pl.Datetime, fmt="%Y-%m-%d").cast(pl.Datetime)
    ])

#print(income_pl)
from datetime import datetime, timedelta

income_pl.filter(
    (pl.col("end") - pl.col("start") >= timedelta(weeks=51)) & 
    (pl.col("frame") != None)
)

start,end,val,accn,fy,fp,form,filed,frame
datetime[μs],datetime[μs],i64,str,i64,str,str,datetime[μs],str
2015-01-01 00:00:00,2015-12-31 00:00:00,-8673000,"""0001564590-18-...",2017,"""FY""","""10-K""",2018-02-27 00:00:00,"""CY2015"""
2016-01-01 00:00:00,2016-12-31 00:00:00,13425000,"""0001564590-19-...",2018,"""FY""","""10-K""",2019-02-26 00:00:00,"""CY2016"""
2017-01-01 00:00:00,2017-12-31 00:00:00,39122000,"""0001564590-20-...",2019,"""FY""","""10-K""",2020-02-25 00:00:00,"""CY2017"""
2018-01-01 00:00:00,2018-12-31 00:00:00,73185000,"""0001564590-21-...",2020,"""FY""","""10-K""",2021-02-16 00:00:00,"""CY2018"""
2019-01-01 00:00:00,2019-12-31 00:00:00,100443000,"""0000950170-22-...",2021,"""FY""","""10-K""",2022-02-15 00:00:00,"""CY2019"""
2020-01-01 00:00:00,2020-12-31 00:00:00,145384000,"""0000950170-22-...",2021,"""FY""","""10-K""",2022-02-15 00:00:00,"""CY2020"""
2021-01-01 00:00:00,2021-12-31 00:00:00,181848000,"""0000950170-22-...",2021,"""FY""","""10-K""",2022-02-15 00:00:00,"""CY2021"""
